<a href="https://colab.research.google.com/github/Priyo-prog/Deep-Learning-with-Tensorflow/blob/main/Natural%20Language%20Processing/NLP_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Introduction

NLP has the goal of derieving information out of natural language (could be sequence text or speech).

Another common terms for NL problems is sequence to sequence problems (seq2seq).

## Check For GPU

In [1]:
!nvidia-smi

Fri Sep 20 00:19:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Get Helper Functions

In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-20 00:19:21--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-09-20 00:19:21 (15.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a Text Dataset

Thedataset we are going to use is Kaggle's intoduction to NLP datset (testx sample of tweets labelled as disaster or not disaster) Binary Classification

In [4]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-09-20 00:19:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.207, 74.125.199.207, 74.125.142.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-09-20 00:19:31 (105 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Import Libraries

In [5]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

## Visualizing Text Dataset

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [8]:
# Shuffle the training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [10]:
# Let's visualize some random training samples
# create random indexes not higher than the total number of samples
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  # First underscore is to skip the index number
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n {text}\n")
  print("-----\n")

Target: 1 (real disaster)
Text:
 Another White mass murderer. #Antioch  https://t.co/OWpd7vcFS6

-----

Target: 0 (not real disaster)
Text:
 Harshness Follows Us a
Better Day
by Sarah C
Racing thoughts with screaming sirens
Pacing back and forth for... http://t.co/ProNtOuo91

-----

Target: 0 (not real disaster)
Text:
 HAPPENING NOW - HATZOLAH EMS AMBULANCE RESPONDING WITH DUAL SIRENS ANDÛ_ https://t.co/SeK6MQ6NJF

-----

Target: 0 (not real disaster)
Text:
 @johngreen storm and silence by @RobThier_EN

-----

Target: 1 (real disaster)
Text:
 #pakistan#news# PAKPATTAN City News: Man electrocuted From Our Correspondent PAKPATTAN: A man was electrocuted... http://t.co/frpbNhVPyI

-----



## Split Data Into Training And Validation Sets

In [11]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(), train_df_shuffled["target"].to_numpy(),
    test_size=0.1, random_state=42)

In [12]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [13]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Change Text to Numbers

**Tokenization** - Straight mapping from token(a token could be aword or character) to number (can be modelled but quickly gets too big). Can be direct mapping like `0,1,2` or `A-1, B-2.... Z-26` or `one hot encoding` in that case it would be too large to handle

**Embedding** - Richer representation of relationships between tokens (can limit size + can be learned). This can update at each epochs of deep neural network learning.

In [14]:
# Use the Default Text Vectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create gropus of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be
                                    pad_to_max_tokens=False)

## Average number of tokens in training tweets

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

## Setup Text Vectorization Variables

In [16]:
max_vocab_length = 10000 # max number of vocab to have in our vocabulary
max_length = 15 # max length our sequences will be(e.g. how many words from a tweet does a model see ?)
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training set
text_vectorizer.adapt(train_sentences)

In [18]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original Sentence \n {random_sentence}\
      \n\nVectorized Version: ")
text_vectorizer([random_sentence])

Original Sentence 
 rgj: Truck trailer catches fire in Reno http://t.co/kAF3WdRmTn      

Vectorized Version: 


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[9101,  259, 1941, 4031,   42,    4, 2320,    1,    0,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all the unique words in the vocabulary
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab : {len(words_in_vocab)}")
print(f"5 most common words : {top_5_words}")
print(f"5 least common words : {bottom_5_words}")

Number of words in vocab : 10000
5 most common words : ['', '[UNK]', 'the', 'a', 'in']
5 least common words : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Create Embedding Using Embedding Layer

To make our embedding we are going to use Tensorflow's embedding layer : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about our embedding layer:
* `input_dim` - The size of our vocabulary
* `output_dim` - the size of the output embedding vector, for example, a value of 100 would mean each tokens gets represented by vector 100 long

* `input_length` - Length of the sequences being passed to the embedding layer

In [21]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, #output_shape
                             input_length=max_length, # how long is each input
                             )
embedding

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original Text :\n{random_sentence}\
       \n\n Embedded version : ")

# Embded the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text :
Tell me why or why not
to adopt in this way
maybe I overlooked something
fatal for me
?whyor why not?       

 Embedded version : 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00935882,  0.01734706, -0.02449628, ...,  0.02450346,
          0.01990856, -0.00759693],
        [-0.02023349,  0.03727334, -0.02557207, ..., -0.01808796,
          0.03480682,  0.0218044 ],
        [-0.03135286,  0.00638204,  0.01418742, ..., -0.01713653,
          0.04295151, -0.01967175],
        ...,
        [-0.04498104,  0.03050008, -0.00744103, ...,  0.03268785,
          0.03287162, -0.02152859],
        [-0.04686438, -0.00636064,  0.02860761, ...,  0.04370824,
         -0.03599533, -0.02604419],
        [ 0.01853759,  0.03658662, -0.02532338, ..., -0.00083183,
          0.01656756,  0.0011248 ]]], dtype=float32)>

In [23]:
# Check out single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00935882,  0.01734706, -0.02449628, -0.00779615, -0.02071811,
        -0.02349008,  0.0275785 , -0.00924914,  0.04497084,  0.03106555,
        -0.02820375, -0.02940832,  0.04746039,  0.04117009, -0.03044888,
         0.0135366 , -0.02930224, -0.00026112,  0.03601846,  0.0343653 ,
         0.01257758, -0.00573979, -0.00533183, -0.04560553,  0.04995639,
         0.01827896, -0.04172751, -0.02427353, -0.02957609, -0.00331058,
        -0.03379176, -0.04971911, -0.03388784, -0.03601276, -0.02117052,
        -0.03682024,  0.02577584, -0.03009882,  0.00675976, -0.04699682,
         0.02715078, -0.01956536,  0.0153981 , -0.02592247, -0.04287268,
         0.01405786,  0.00556573,  0.01333519, -0.03371841,  0.04412944,
         0.03586088, -0.04833065, -0.0265837 , -0.04276003,  0.0226096 ,
         0.04775857, -0.00334474,  0.00853552, -0.02263445,  0.02853626,
        -0.0381599 , -0.04858059,  0.00583012, -0.00968118, -0.00141567,
  

## Modelling a dataset (running a set of experiments)

Now we've a got way to turn out text sequences into numbers it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn-map
* Model 1: Feed-Forward Neural Network (Dense Model)
* Model 2: LSTM model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

**Approach :**

* Create a model
* Build a model
* Fit a model
* Evaluate our model




## Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create baseline model so you've got a benchmark for future experiments to build upon.

To crate our baseline, we'll use Sklearn's Multinomial Naive Bayes using TF-IDF Formula to convert our words to numbers.

**Note** : It's common use to non-DL algorithms as a baseline because of their speed and then later using DL to see if you improve upon them.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create Tokenization and modelling pipeline
model_0 = Pipeline(
    steps=[("ifidf", TfidfVectorizer()), # convert numbers into words using tfidf
           ("clf", MultinomialNB()) # Model The text
           ])

# Fit The pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('ifidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
## Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
train_df["target"].value_counts()

,count
target,
0,4342
1,3271


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating Evaluation Function for Model Experiments

In [28]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
   Calculates model accuracy, recall, precision and f1 score
   for a binary classification model
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [29]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [31]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

## Create Model With Functional API

In [51]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # Inputs are 1-dimensional String
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)  # Flatten the embedding output
outputs = layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(x) # Create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [48]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 15, 1)               │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
# compile the model
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [54]:
# fit the model
model_1_history = model_1.fit(x=train_sentences,
                               y=train_labels,
                               epochs=5,
                              validation_data=(val_sentences, val_labels),
                               callbacks=[create_tensorboard_callback(dir_name="SAVE_DIR",
                                                                      experiment_name="model_1_dense")])

Saving TensorBoard log files to: SAVE_DIR/model_1_dense/20240920-010434
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6680 - loss: 0.6055 - val_accuracy: 0.7835 - val_loss: 0.4944
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8892 - loss: 0.3601 - val_accuracy: 0.7887 - val_loss: 0.4612
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9020 - loss: 0.2801 - val_accuracy: 0.7900 - val_loss: 0.4638
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9151 - loss: 0.2367 - val_accuracy: 0.7835 - val_loss: 0.4797
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9263 - loss: 0.2025 - val_accuracy: 0.7848 - val_loss: 0.4995


## Check The Model Predictions

In [44]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7737 - loss: 0.5100


[0.47774195671081543, 0.7860892415046692]

In [45]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


(762, 1)

If we didn't put the GlobalAveragePooling1D for flatenning the layer then prediction would have come for all the tokens rather than the entire sample sentence

In [55]:
model_1_pred_probs[0]

array([0.42365885], dtype=float32)

In [56]:
# Look at The first 10 predictions
model_1_pred_probs[:10]

array([[0.42365885],
       [0.8281021 ],
       [0.9979247 ],
       [0.10248885],
       [0.1185339 ],
       [0.93887705],
       [0.9254779 ],
       [0.9926884 ],
       [0.9655366 ],
       [0.23924714]], dtype=float32)

## Compare Predictions With Labels

In [58]:
 # Convert model prediction probabilities to label format
 model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
 model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

## Classification result comparison of Labels and Predictions

In [59]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7899352475221245,
 'recall': 0.7860892388451444,
 'f1': 0.7834372696728737}